# **PREDICTION**

In [ ]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# **LOADING MODEL**

In [ ]:
model = lgb.Booster(model_file='/home/014491542/CMPE-256 Project/Models/LightGBM-v2.lgb')

In [ ]:
df = pd.read_csv('/home/014491542/CMPE-256 Project/DataSet/validationDataset.csv')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,lag_21,lag_28,rolling_ave_7_7,rolling_ave_14_14,rolling_ave_21_21,rolling_ave_28_28,quarter,mday,is_month_start,is_month_end
0,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,d_1914,0.0,2016-04-25,11613,...,0.0,1.0,0.857143,0.928571,0.952381,1.035714,2,25,0,0
1,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,d_1915,0.0,2016-04-26,11613,...,4.0,0.0,1.000000,0.928571,1.095238,0.964286,2,26,0,0
2,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,d_1916,0.0,2016-04-27,11613,...,2.0,0.0,1.142857,0.928571,1.142857,0.964286,2,27,0,0
3,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,d_1917,2.0,2016-04-28,11613,...,3.0,0.0,1.428572,1.000000,1.238095,0.928571,2,28,0,0
4,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,d_1918,0.0,2016-04-29,11613,...,0.0,0.0,1.285714,1.071428,1.095238,0.892857,2,29,0,0


In [ ]:
to_predict = df.drop(columns=['id','d','sales','date','wm_yr_wk','weekday','wday','is_month_start','is_month_end'])

In [ ]:
to_predict.head()

,item_id,dept_id,store_id,cat_id,state_id,month,year,event_name_1,event_type_1,event_name_2,...,lag_7,lag_14,lag_21,lag_28,rolling_ave_7_7,rolling_ave_14_14,rolling_ave_21_21,rolling_ave_28_28,quarter,mday
0,0,0,0,0,0,4,2016,0,0,0,...,1.0,0.0,0.0,1.0,0.857143,0.928571,0.952381,1.035714,2,25
1,0,0,0,0,0,4,2016,0,0,0,...,1.0,0.0,4.0,0.0,1.000000,0.928571,1.095238,0.964286,2,26
2,0,0,0,0,0,4,2016,0,0,0,...,1.0,0.0,2.0,0.0,1.142857,0.928571,1.142857,0.964286,2,27
3,0,0,0,0,0,4,2016,0,0,0,...,3.0,1.0,3.0,0.0,1.428572,1.000000,1.238095,0.928571,2,28
4,0,0,0,0,0,4,2016,0,0,0,...,0.0,1.0,0.0,0.0,1.285714,1.071428,1.095238,0.892857,2,29


# **MAKE PREDICTIONS**

In [ ]:
to_predict['sales_pred'] = model.predict(to_predict)
to_predict['sales_orig'] = df['sales']

In [ ]:
cols = [f"F{i}" for i in range(1,29)]
predictionsDF = pd.DataFrame(columns=['F','id']+cols)

In [ ]:
predictionsDF['F']=[i for i in range(len(to_predict['id'].unique()))]
predictionsDF.head()

,F,id,F1,F2,F3,F4,F5,F6,F7,F8,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **PREDICTION DATAFRAME FOR NEXT 28 DAYS**

In [ ]:
row = 0
for ID in to_predict.id.unique():
    predictionsDF['id'][row]=ID
    day = 1
    for val in to_predict[to_predict['id']==ID]['sales_pred']:
        predictionsDF['F'+str(day)][row] = np.round(val,2)
        day += 1
    row += 1

In [ ]:
predictionsDF.head()

,F,id,F1,F2,F3,F4,F5,F6,F7,F8,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,0,HOBBIES_1_001_CA_1_evaluation,0.91,0.96,0.96,1.21,0.94,1.04,1.16,0.97,...,0.91,1.21,1.07,1.04,1.04,0.99,1.02,0.98,1.34,1.06
1,1,HOBBIES_1_002_CA_1_evaluation,0.18,0.17,0.3,0.25,0.25,0.22,0.26,0.22,...,0.17,0.17,0.16,0.18,0.19,0.21,0.16,0.15,0.18,0.26
2,2,HOBBIES_1_003_CA_1_evaluation,0.57,0.6,0.62,0.57,0.58,0.5,0.65,0.48,...,0.62,0.64,0.69,0.54,0.52,0.59,0.54,0.62,0.56,0.72
3,3,HOBBIES_1_004_CA_1_evaluation,1.87,1.78,1.73,1.64,2.01,2.22,2.69,2.05,...,2.08,2.06,2.51,2.23,1.75,1.68,1.73,1.98,1.87,1.75
4,4,HOBBIES_1_005_CA_1_evaluation,1.06,1.06,1.2,1.24,1.17,1.08,1.66,1.24,...,1.45,1.36,1.65,1.52,1.56,1.51,1.66,1.45,1.35,1.49


# **ORIGINAL DATAFRAME FOR NEXT 28 DAYS**

In [ ]:
originalsDF = pd.DataFrame(columns=['F','id']+cols)
originalsDF['F']=[i for i in range(len(to_predict['id'].unique()))]

row = 0
for ID in to_predict.id.unique():
    originalsDF['id'][row]=ID
    day = 1
    for val in to_predict[to_predict['id']==ID]['sales_orig']:
        originalsDF['F'+str(day)][row] = val
        day += 1
    row += 1

In [ ]:
originalsDF.head()

,F,id,F1,F2,F3,F4,F5,F6,F7,F8,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,0,HOBBIES_1_001_CA_1_evaluation,0,0,0,2,0,3,5,0,...,2,4,0,0,0,0,3,3,0,1
1,1,HOBBIES_1_002_CA_1_evaluation,0,1,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,2,HOBBIES_1_003_CA_1_evaluation,0,0,1,1,0,2,1,0,...,1,0,2,0,0,0,2,3,0,1
3,3,HOBBIES_1_004_CA_1_evaluation,0,0,1,2,4,1,6,4,...,1,1,0,4,0,1,3,0,2,6
4,4,HOBBIES_1_005_CA_1_evaluation,1,0,2,3,1,0,3,2,...,0,0,0,2,1,0,0,2,1,0


# **R2 SCORE**

In [ ]:
from sklearn.metrics import r2_score
r2_score(to_predict['sales_orig'],to_predict['sales_pred'])

0.6763236753682769